# install 목록

In [ ]:
!pip install opencv-python
!pip install dlib
!pip install numpy
!pip install tensorflow
!pip install keras

# import 먼저 해보고 numpy 다운그레이드 오류가 생길시 numpy삭제후 다음 실행

In [10]:
!pip uninstall numpy
!pip install "numpy<2"

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Would remove:
    /opt/homebrew/bin/f2py
    /opt/homebrew/lib/python3.12/site-packages/numpy-1.26.4.dist-info/*
    /opt/homebrew/lib/python3.12/site-packages/numpy/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


# import

In [1]:
import cv2
import dlib
import numpy as np
from keras.models import load_model

# main code

### 사전 데이터 로드

In [2]:
# 얼굴 인식
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# 표정 인식을 위한 눈, 코, 입등의 위치 반환
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# 표정 라벨링
expression_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# 표정 가중치 모델
model = load_model('emotion_model.hdf5', compile=False)


### 필터 클래스 추가 ( 여기서 작업하시면 될 것 같아요)

In [6]:
class EmotionFilters:
    def __init__(self):
        # 각 표정에 해당하는 필터를 미리 준비해 놓을 수 있음
        self.filters = {
            'Angry': self.angry_filter,
            'Disgust': self.disgust_filter,
            'Fear': self.fear_filter,
            'Happy': self.happy_filter,
            'Sad': self.sad_filter,
            'Surprise': self.surprise_filter,
            'Neutral': self.neutral_filter
        }
    
    def apply_filter(self, frame, expression_label):
        # 표정에 맞는 필터 적용
        if expression_label in self.filters:
            frame = self.filters[expression_label](frame)
        return frame
    
    def angry_filter(self, frame):
    # 예시 필터: 색상 강조
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        frame[:, :, 1] = 255  # 채도 증가
        frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
        return frame
    
    # 각 표정에 맞는 필터 함수들을 정의 (구조만 남김)
    def disgust_filter(self, frame):
        # 표정에 맞는 필터 적용
        return frame
    
    def fear_filter(self, frame):
        # 표정에 맞는 필터 적용
        return frame
    
    def happy_filter(self, frame):
        # 표정에 맞는 필터 적용
        return frame
    
    def sad_filter(self, frame):
        # 표정에 맞는 필터 적용
        return frame
    
    def surprise_filter(self, frame):
        # 표정에 맞는 필터 적용
        return frame
    
    def neutral_filter(self, frame):
        # 표정에 맞는 필터 적용
        return frame


## 비디오 캠 버전 - 표정 인식 연습용

In [8]:
# 얼굴 인식
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# 표정 인식을 위한 눈, 코, 입등의 위치 반환
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# 표정 라벨링
expression_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# 표정 가중치 모델
model = load_model('emotion_model.hdf5', compile=False)
emotion_filters = EmotionFilters()

# 비디오 실행
video_capture = cv2.VideoCapture(0)

prev_faces = []

while True:
    # ret, frame 반환
    ret, frame = video_capture.read()
    
    if not ret:
        break

    # 얼굴인식을 위해 gray 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 인식
    # scaleFactor이 1에 가까울수록 표정 인식이 잘 되고 멀 수록 잘 안됨
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    #region 얼굴이 인식되면 표정을 인식
    for (x, y, w, h) in faces:
        # 얼굴 크기에 알맞도록 사각형 그리기
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # 얼굴 크기 반환
        face_roi = gray[y:y+h, x:x+w]

        # 표정을 인식하기 위해 표정 dataset과 똑같은 사이즈 변환
        # dataset 이미지와 입력된 얼굴의 크기가 다르면 error 발생
        face_roi = cv2.resize(face_roi, (64, 64))
        face_roi = np.expand_dims(face_roi, axis=-1)
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = face_roi / 255.0

        # 모델을 통해 표정 분석
        output = model.predict(face_roi)[0]

        # 해당 표정의 값 반환
        expression_index = np.argmax(output)

        # 표정에 따른 label 값 저장
        expression_label = expression_labels[expression_index]
           # 표정에 맞는 필터 적용
        frame = emotion_filters.apply_filter(frame, expression_label)
        # print(expression_label, end=' ')
        # 표정 값 출력
        cv2.putText(frame, expression_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    #endregion
    
    # 출력
    cv2.imshow('Expression Recognition', frame)

    # esc 누를 경우 종료
    key = cv2.waitKey(25)
    if key == 27:
        break

video_capture.release()
cv2.destroyAllWindows()
# 밑에는 오류가 아닌 창이 안닫힐때 해결법 
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


-1

## 영상인식 버전 - 제출할 코드용

In [9]:
# 필터 클래스 초기화
emotion_filters = EmotionFilters()

prev_faces = []
# 저장된 비디오 파일 경로로 변경
video_capture = cv2.VideoCapture('example/emotion.mp4')  # 비디오 파일 경로 변경
while True:
    # ret, frame 반환
    ret, frame = video_capture.read()
    
    if not ret:
        break

    # 얼굴인식을 위해 gray 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 인식
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        # 얼굴 크기에 알맞도록 사각형 그리기
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # 얼굴 크기 반환
        face_roi = gray[y:y+h, x:x+w]

        # 표정을 인식하기 위해 표정 dataset과 똑같은 사이즈 변환
        face_roi = cv2.resize(face_roi, (64, 64))
        face_roi = np.expand_dims(face_roi, axis=-1)
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = face_roi / 255.0

        # 모델을 통해 표정 분석
        output = model.predict(face_roi)[0]

        # 해당 표정의 값 반환
        expression_index = np.argmax(output)

        # 표정에 따른 label 값 저장
        expression_label = expression_labels[expression_index]
        
        # 표정에 맞는 필터 적용
        frame = emotion_filters.apply_filter(frame, expression_label)

        # 표정 라벨 화면에 표시
        cv2.putText(frame, expression_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # 출력
    cv2.imshow('Expression Recognition with Filters', frame)

    # esc 누를 경우 종료
    key = cv2.waitKey(1)  # 대기 시간을 1ms로 설정
    if key == 27:  # ESC 키가 눌리면 종료
        break

video_capture.release()
cv2.destroyAllWindows()
# 맥의 경우 아래 waitkey를 반복해야 꺼짐 
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

-1